In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
import time
from bs4 import BeautifulSoup

In [1]:
url = "https://www.premierleague.com/results"

In [4]:
options = webdriver.ChromeOptions()
options.set_capability('goog:loggingPrefs', {"performance": "ALL", "browser": "ALL"})
# Adding user agent to look more like a real browser
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36")

driver = webdriver.Chrome(options=options)
wait = WebDriverWait (driver,10)
driver.set_page_load_timeout(50)
driver.set_window_size(800, 600)

In [5]:
driver.get(url)
time.sleep(3)  # Allow page to load


# Scroll to load all past matches
last_height = driver.execute_script("return document.documentElement.scrollHeight")
print ("last height: " , last_height)
prev = 0
while True:
    next_height = 0
    while next_height < last_height - 200:
        next_height = prev + 25
        driver.execute_script(f"window.scrollTo({prev}, {next_height});")
        time.sleep (0.1)
        prev = next_height
    # Wait for new matches to load
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    print ("new_height: ", new_height)
    if new_height == last_height:
        break  # Stop when no more matches load
    last_height = new_height

time.sleep (5) # time to load
soup = BeautifulSoup(driver.page_source, "html.parser")
season_matches = soup
driver.quit()  # Close Selenium

last height:  3682
new_height:  6207
new_height:  8758
new_height:  11206
new_height:  13601
new_height:  16152
new_height:  18600
new_height:  20944
new_height:  23194
new_height:  23158
new_height:  23158


In [11]:
date_containers = season_matches.find_all('div', 'fixtures__date-container')
match_links = []

for container in date_containers:
    for match_fixtures in container.find_all('li', class_='match-fixture'):
        match_links.append(match_fixtures.get('data-comp-match-item'))

In [17]:
print (len(set(match_links)))

338


In [18]:
with open("match_id.csv", "a") as f:
    f.write("\n".join(match_links) + "\n")